In [11]:
using NBInclude
@nbinclude("dcj_datastructs.ipynb")
@nbinclude("helpers.ipynb")

update_adj_set (generic function with 1 method)

In [13]:
function reassign_locs(gid_to_l::DefaultDict{Int, Vector{Int}}, adj_list, u_idx, v_idx)
    for (i, adj) in pairs(view(adj_list, min(u_idx, v_idx):max(u_idx, v_idx)))
        left = adj.left 
        right = adj.right 

        idx = Ref{Int}(i + min(u_idx, v_idx) - 1)

        process_adj_list_helper(left, idx, gid_to_l)
        process_adj_list_helper(right, idx, gid_to_l)
    end 
end 

update_adj_set (generic function with 1 method)

In [16]:
function update_documentation(u_idx::Int, v_idx::Int, pq::Adjacency, excluding_pq::Adjacency, src_adjs::Vector{Adjacency}, src_gid_to_l::DefaultDict{Int, Vector{Int}})
    # update src adjs
    src_adjs[u_idx] = pq 
    src_adjs[v_idx] = excluding_pq 

    # update geneend locations # TODO 
    # reassign_locs(gid_to_l, adj_list, u_idx, v_idx)
end 


# find dcj operations and distance
function find_dcj_dist_ops(src_adjs::Vector{Adjacency}, tar_adj_list::Vector{Adjacency}, src_gid_to_l::DefaultDict{Int, Vector{Int}}, tar_gid_to_l::DefaultDict{Int, Vector{Int}}, src_adj_set::Set{Adjacency}, target_adj_set::Set{Adjacency})
    count = 0  
    telomere_idxs = Vector{Int}()
    updated_adj_list = src_adjs

    print("indexing through target adj list...")
    
    # for each adj {p, q} in target genome 
    for (i, adj) in pairs(tar_adj_list) 
        print("\n\n")
        show(updated_adj_list)
         
        print("\nADJ ", i, " :::: ")
        show(adj, false)
        print("\n")

        p = adj.left  
        q = adj.right  

        if p.gene == Telomere() || q.gene == Telomere()  # telomeres handled in next loop
            push!(telomere_idxs, i)  
        else 
            #  let u be adj of genome A that contains p 
            #  let v be adj of genome A that contains q 
            u, u_idx = find_tar_ge_in_src_adjs(p, src_gid_to_l, tar_gid_to_l, src_adjs)  
            v, v_idx = find_tar_ge_in_src_adjs(q, src_gid_to_l, tar_gid_to_l, src_adjs)  
            
            #  if u != v, replace u and v in A by {p, q} and (u\{p}) U (v\{q})
            if u_idx != v_idx             
                pq, excluding_pq = update_adj_set(p, q, u, v, u_idx, v_idx, src_adjs, src_adj_set)
            
                print("updated adj set ::: ")
                show(src_adj_set)
                print("\nupdated adj list ::: ")
                updated_adj_list = convert_adj_set_as_adj_list(src_adj_set)
                show(updated_adj_list, pq, excluding_pq)

                update_documentation(u_idx, v_idx, pq, excluding_pq, src_adjs, src_gid_to_l)
                
                println("\n\nupdated src_adjs")
                show(src_adjs)
                println("\nupdated src_gid_to_location")
                show(src_gid_to_l)

                count += 1 

                break
            end   
        end
    end 
    # for each telomere {p} in genome B 
    # for idx in telomere_idxs
    #     telo_adj = tar_adj_list[idx]
    #     telo_ge = other_adjacency_end(telo_adj, Telomere())

    #     #  let u be element of genome A that contains p 
    #     u, u_idx = find_tar_ge_in_src_adjlist(p, src_gid_to_l, tar_gid_to_l, src_adjs)  

    #     #  if u is an adjacency, then replace u in A by {p} and (u\{p})
    #     if other_adjacency_end(tar_adj_list[u_idx], u) != Telomere() 
    #         # update_adj_set(p, u, u_idx,src_adjs, src_adj_set)
    #         # update_adj_list_gid_loc_dict()

    #         count += 1 
    #     end 
    # end 
    return count 
end 

src = "abc"
target = "bac"

calculate_distance(src, target)

SRC ADJ LIST
(a:h,b:t)(b:h,c:t)(c:h,a:t)
|
v
TARGET ADJ LIST
(b:h,a:t)(a:h,c:t)(c:h,b:t)

*************
SRC GENE --> LOCATION DICT
a 2 => [3, 1]
c 3 => [2, 3]
b 1 => [1, 2]


TARGET GENE --> LOCATION DICT
a 2 => [1, 2]
c 3 => [2, 3]
b 1 => [3, 1]

*************

indexing through target adj list...

(a:h,b:t)(b:h,c:t)(c:h,a:t)
ADJ 1 :::: (b:h,a:t)
updated adj set ::: (a:h,b:t)(c:t,c:h)(b:h,a:t)
updated adj list ::: (a:h,b:t)(b:h,a:t)(c:t,c:h)

updated src_adjs
(a:h,b:t)(b:h,a:t)(c:t,c:h)
updated src_gid_to_location
DefaultDict(2 => [3, 1], 3 => [2, 3], 1 => [1, 2])

1